In [ ]:
pip install qiskit

In [ ]:
pip install qiskit-aer

## Setting up a simple quantum circuit

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(1, 1)
qc.h(0) 
qc.save_density_matrix()  
print(qc)


     ┌───┐ density_matrix 
  q: ┤ H ├───────░────────
     └───┘       ░        
c: 1/═════════════════════
                          


## Ideal state

In [25]:
from qiskit_aer import AerSimulator
from qiskit import transpile

ideal_backend_dm = AerSimulator(method="density_matrix")
qc_t = transpile(qc, ideal_backend_dm)

job = ideal_backend_dm.run(qc_t)
result = job.result()

rho_ideal = result.data(0)['density_matrix']
print(rho_ideal)


DensityMatrix([[0.5+0.j, 0.5+0.j],
               [0.5+0.j, 0.5+0.j]],
              dims=(2,))


In the density matrix the:
- Diagonals → probabilities
- Off-diagonals → coherence (alive and well)

This is a pure quantum state.

#### Measuring probabilities 

In [26]:
qc_meas = qc.copy()
qc_meas.measure_all()

ideal_backend_counts = AerSimulator()
qc_t = transpile(qc_meas, ideal_backend_counts)

job = ideal_backend_counts.run(qc_t, shots=1000)
result = job.result()

counts_ideal = result.get_counts()
print(counts_ideal)


{'0 0': 499, '1 0': 501}


## Noisy state

In [28]:
from qiskit_aer.noise import NoiseModel, phase_damping_error

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(
    phase_damping_error(0.4),
    ['h']
)

noisy_backend_dm = AerSimulator(
    method="density_matrix",
    noise_model=noise_model
)

qc_t = transpile(qc, noisy_backend_dm)

job = noisy_backend_dm.run(qc_t)
result = job.result()

rho_noisy = result.data(0)['density_matrix']
print(rho_noisy)

DensityMatrix([[0.5       +0.j, 0.38729833+0.j],
               [0.38729833+0.j, 0.5       +0.j]],
              dims=(2,))


In [29]:
qc_meas = qc.copy()
qc_meas.measure_all()

noisy_backend_counts = AerSimulator(noise_model=noise_model)
qc_t = transpile(qc_meas, noisy_backend_counts)

job = noisy_backend_counts.run(qc_t, shots=1000)
result = job.result()

counts_noisy = result.get_counts()
print(counts_noisy)


{'1 0': 513, '0 0': 487}


Here is what we see :
- Probabilities remain unchanged
- Coherence has decayed
- This means superposition is no longer usable hence making it equivalent to a classical state
<br>

|      Case          |Density matrix                          |Counts                         |
|----------------|-------------------------------|-----------------------------|
|Ideal|Coherent           |50/50            |
|Noisy          |     Decohered      |Still 50/50           |

<i>Decoherence happened, but counts didn’t change.</i> <br>

What just happened conceptually:

- The system lost phase information
- But phase doesn’t affect probabilities
- So measurement statistics look fine
- But interference is gone forever

That’s why quantum computers fail silently.<br>
<b>Decoherence removes the ability to interfere, not the randomness of outcomes.</b>
